## 1) 신경망

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
seed = 2022
np.random.seed(seed)
tf.random.set_seed(seed)

In [3]:
from tensorflow.keras.datasets import fashion_mnist
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

4431872/4422102 [==============================] - 0s 0us/step


In [4]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
class_names[y_train[0]]

'Ankle boot'

In [5]:
X_train.shape, X_test.shape

((60000, 28, 28), (10000, 28, 28))

In [6]:
# X data 정규화 및 차원 변경
X_train = X_train.reshape(60000,-1) / 255.
X_test = X_test.reshape(-1,784) / 255.
X_train.shape, X_test.shape

((60000, 784), (10000, 784))

In [7]:
# y data : one-hot encoding
from tensorflow.keras.utils import to_categorical
Y_train = to_categorical(y_train)
Y_test = to_categorical(y_test)
Y_train.shape, Y_test.shape

((60000, 10), (10000, 10))

* 모델 정의, 설정, 학습, 평가

In [8]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [9]:
model = Sequential([
    Dense(512,input_dim=784, activation='relu'),
    Dense(48, activation='relu'),
    Dense(10, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               401920    
                                                                 
 dense_1 (Dense)             (None, 48)                24624     
                                                                 
 dense_2 (Dense)             (None, 10)                490       
                                                                 
Total params: 427,034
Trainable params: 427,034
Non-trainable params: 0
_________________________________________________________________


In [10]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_path= 'mnist_dnn.h5'
checkpoint = ModelCheckpoint(model_path, save_best_only=True, verbose=1)
early_stop = EarlyStopping(patience=10)

In [11]:
hist = model.fit(X_train, Y_train , validation_split=0.2,
                 epochs=100, batch_size=200, verbose=0,
                 callbacks=[checkpoint,early_stop])


Epoch 00001: val_loss improved from inf to 0.42591, saving model to mnist_dnn.h5

Epoch 00002: val_loss improved from 0.42591 to 0.38297, saving model to mnist_dnn.h5

Epoch 00003: val_loss improved from 0.38297 to 0.34602, saving model to mnist_dnn.h5

Epoch 00004: val_loss improved from 0.34602 to 0.33033, saving model to mnist_dnn.h5

Epoch 00005: val_loss did not improve from 0.33033

Epoch 00006: val_loss improved from 0.33033 to 0.32579, saving model to mnist_dnn.h5

Epoch 00007: val_loss improved from 0.32579 to 0.30991, saving model to mnist_dnn.h5

Epoch 00008: val_loss did not improve from 0.30991

Epoch 00009: val_loss improved from 0.30991 to 0.30931, saving model to mnist_dnn.h5

Epoch 00010: val_loss improved from 0.30931 to 0.30319, saving model to mnist_dnn.h5

Epoch 00011: val_loss did not improve from 0.30319

Epoch 00012: val_loss did not improve from 0.30319

Epoch 00013: val_loss improved from 0.30319 to 0.30046, saving model to mnist_dnn.h5

Epoch 00014: val_loss

In [12]:
best_model = load_model(model_path)
best_model.evaluate(X_test, Y_test)

313/313 [==============================] - 1s 3ms/step - loss: 0.3256 - accuracy: 0.8886


[0.32563674449920654, 0.8885999917984009]

## 2) SVM

In [13]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

In [14]:
params = {'C': [0.1, 1, 5, 10]}
svc = SVC(random_state=2022)

In [15]:
grid_svc = GridSearchCV(
    svc, param_grid=params, scoring='accuracy', cv=3, n_jobs=-1
)
%time grid_svc.fit(X_train, y_train)

CPU times: user 5min 31s, sys: 3.94 s, total: 5min 35s
Wall time: 1h 14min 11s


GridSearchCV(cv=3, estimator=SVC(random_state=2022), n_jobs=-1,
             param_grid={'C': [0.1, 1, 5, 10]}, scoring='accuracy')

In [17]:
grid_svc.best_params_

{'C': 10}

In [18]:
grid_svc.best_estimator_.score(X_test, y_test)

0.9002

## 3) CNN v4
* 2 Conv2D layers ==> 2Conv2D with padding layers
* 1 MaxPoling2D layer
* 1 Dropout(0.25) layer
* 1 Fully Connected Network layer
* 1 Dropout(0.5) layer
* 1 Output layer

In [20]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [27]:
from tensorflow.keras.datasets import fashion_mnist
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

In [28]:
# X data: 0 ~ 1 사이의 값, 3차원 --> 4차원
X_train = X_train.reshape(-1, 28, 28, 1) / 255.
X_test = X_test.reshape(-1,28,28,1) / 255.

In [29]:
# One-hot encoding
from tensorflow.keras.utils import to_categorical
Y_train = to_categorical(y_train)
Y_test = to_categorical(y_test)

In [31]:
model = Sequential([
    Conv2D(32, kernel_size=(3,3), padding='same', input_shape=(28,28,1), activation='relu'),
    Conv2D(64,(3,3), padding='same', activation='relu'),
    MaxPooling2D(),
    Dropout(0.25),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 28, 28, 32)        320       
                                                                 
 conv2d_5 (Conv2D)           (None, 28, 28, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 14, 14, 64)       0         
 2D)                                                             
                                                                 
 dropout_4 (Dropout)         (None, 14, 14, 64)        0         
                                                                 
 flatten_2 (Flatten)         (None, 12544)             0         
                                                                 
 dense_7 (Dense)             (None, 512)               6423040   
                                                      

In [32]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [33]:
model_path = 'best-mnist-cnn-v4.h5'
checkpoint = ModelCheckpoint(model_path, save_best_only=True, verbose=1)
early_stop = EarlyStopping(patience=10)

In [34]:
hist = model.fit(
    X_train, Y_train, validation_split=0.2, epochs=100, batch_size=200, 
    callbacks=[checkpoint, early_stop], verbose=0
)


Epoch 00001: val_loss improved from inf to 0.31710, saving model to best-mnist-cnn-v4.h5

Epoch 00002: val_loss improved from 0.31710 to 0.26670, saving model to best-mnist-cnn-v4.h5

Epoch 00003: val_loss improved from 0.26670 to 0.23533, saving model to best-mnist-cnn-v4.h5

Epoch 00004: val_loss improved from 0.23533 to 0.21617, saving model to best-mnist-cnn-v4.h5

Epoch 00005: val_loss improved from 0.21617 to 0.21492, saving model to best-mnist-cnn-v4.h5

Epoch 00006: val_loss improved from 0.21492 to 0.20709, saving model to best-mnist-cnn-v4.h5

Epoch 00007: val_loss improved from 0.20709 to 0.19675, saving model to best-mnist-cnn-v4.h5

Epoch 00008: val_loss did not improve from 0.19675

Epoch 00009: val_loss improved from 0.19675 to 0.19471, saving model to best-mnist-cnn-v4.h5

Epoch 00010: val_loss did not improve from 0.19471

Epoch 00011: val_loss did not improve from 0.19471

Epoch 00012: val_loss did not improve from 0.19471

Epoch 00013: val_loss did not improve from 

In [35]:
best_model = load_model(model_path)
best_model.evaluate(X_test,Y_test)

313/313 [==============================] - 1s 3ms/step - loss: 0.2154 - accuracy: 0.9278


[0.2153996378183365, 0.9277999997138977]

## 신경망 : 0.8886, SVM : 0.9002, CNN : 0.9277
* 결과에 비추어 보아, CNN의 성능이 월등히 좋다는게 확인되었다. 단 의문점은, 어째서 SVM은 신경망보다 더 좋은 결과를 내는 것일까? 결과는 신경망모델보다 더 좋으나 SVM이 시간이 오래걸리니 효율성 면에선 떨어진다. 출력 속도도 빠르고, 결과도 좋은 CNN일 따름이다. CNN의 모델을 이것저것 만져서 성능을 더 높이도록 해보겠다.

## 4) CNN의 성능 높이기

In [42]:
model2 = Sequential([
    Conv2D(32, kernel_size=(3,3), padding='same', input_shape=(28,28,1), activation='relu'),
    Conv2D(64,(3,3), padding='same', activation='relu'),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
])
model2.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 28, 28, 32)        320       
                                                                 
 conv2d_9 (Conv2D)           (None, 28, 28, 64)        18496     
                                                                 
 flatten_4 (Flatten)         (None, 50176)             0         
                                                                 
 dense_12 (Dense)            (None, 512)               25690624  
                                                                 
 dropout_9 (Dropout)         (None, 512)               0         
                                                                 
 dense_13 (Dense)            (None, 256)               131328    
                                                                 
 dropout_10 (Dropout)        (None, 256)              

In [43]:
model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [44]:
model2_path = 'best-mnist-cnn-v4.h5'
checkpoint = ModelCheckpoint(model_path, save_best_only=True, verbose=1)
early_stop = EarlyStopping(patience=10)

In [45]:
hist = model2.fit(
    X_train, Y_train, validation_split=0.2, epochs=100, batch_size=200, 
    callbacks=[checkpoint, early_stop], verbose=0
)


Epoch 00001: val_loss improved from inf to 0.30945, saving model to best-mnist-cnn-v4.h5

Epoch 00002: val_loss improved from 0.30945 to 0.26854, saving model to best-mnist-cnn-v4.h5

Epoch 00003: val_loss improved from 0.26854 to 0.24164, saving model to best-mnist-cnn-v4.h5

Epoch 00004: val_loss improved from 0.24164 to 0.22108, saving model to best-mnist-cnn-v4.h5

Epoch 00005: val_loss did not improve from 0.22108

Epoch 00006: val_loss did not improve from 0.22108

Epoch 00007: val_loss did not improve from 0.22108

Epoch 00008: val_loss did not improve from 0.22108

Epoch 00009: val_loss did not improve from 0.22108

Epoch 00010: val_loss did not improve from 0.22108

Epoch 00011: val_loss did not improve from 0.22108

Epoch 00012: val_loss did not improve from 0.22108

Epoch 00013: val_loss did not improve from 0.22108

Epoch 00014: val_loss did not improve from 0.22108


In [46]:
best_model = load_model(model_path)
best_model.evaluate(X_test,Y_test)

313/313 [==============================] - 1s 4ms/step - loss: 0.2368 - accuracy: 0.9158


[0.23683464527130127, 0.9157999753952026]